In [1]:
!pip install transformers
!pip install accelerate -U
!pip install datasets
!pip install torch-summary
!pip install graphviz
!pip install torchview
!pip install bertviz

repo_path = "https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/"
branch = "lexicon"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 79.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.29.164 which is incompatible.


In [2]:
utils_url = f"{repo_path}{branch}/utils.py"
evaluation_url = f"{repo_path}{branch}/evaluation.py"

import os
if os.path.exists("utils.py"):
  !rm "utils.py"
if os.path.exists("evaluation.py"):
  !rm "evaluation.py"

!wget {utils_url}
!wget {evaluation_url}

--2023-06-30 15:28:41--  https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/lexicon/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14111 (14K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  13.78K  --.-KB/s    in 0s      

2023-06-30 15:28:42 (69.4 MB/s) - ‘utils.py’ saved [14111/14111]

--2023-06-30 15:28:42--  https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/lexicon/evaluation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10675 (10K) [t

In [3]:
import json
import torch
import pandas as pd
import numpy as np
from transformers import BertModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertPreTrainedModel, BertForSequenceClassification
from transformers import TrainingArguments, Trainer, TrainerCallback, EarlyStoppingCallback, PretrainedConfig
from transformers.modeling_outputs import SequenceClassifierOutput
from torch.nn import BCEWithLogitsLoss
import importlib
import sys
from utils import *
importlib.reload(sys.modules['utils'])

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

<module 'utils' from '/kaggle/working/utils.py'>

In [4]:
# set CUDA if available
"""if torch.cuda.is_available():
    device = torch.device('cuda')
    print("======= CUDA Available =======")
else:
    device = torch.device('cpu')
    print("======= CUDA NOT Available, run on CPU =======")"""
device = torch.device('cpu') # otw goes out of memory

In [5]:
NUM_LABELS = 8

model_config = {
    'model_id': 'bert_custom_baseline',
    'tokenizer_name': 'bert-base-cased',
    'model_name': 'bert-base-cased',
    'train_batch_size': 4,
    'val_batch_size': 4,
    'learning_rate': 2e-5,
    'weight_decay': 0.01,
    'epochs': 1,
    'seed': 42,
    'patience': 3,
    'early_stopping_threshold': 0,
} # TODO: expand...

tokenizer = AutoTokenizer.from_pretrained(model_config['tokenizer_name'], truncation=True)

In [6]:
TRAIN_DATA = f"{repo_path}{branch}/datasets/internal_train_essay_level_preproc.tsv"
VAL_DATA = f"{repo_path}{branch}/datasets/internal_val_essay_level_preproc.tsv"
DEV_DATA = f"{repo_path}{branch}/datasets/dev_essay_level_preproc.tsv"

train_df = pd.read_csv(TRAIN_DATA, sep='\t')
val_df = pd.read_csv(VAL_DATA, sep='\t')
dev_df = pd.read_csv(DEV_DATA, sep='\t')

# TODO: come trattare unkown? inferirli?
# per adesso gli unknown in gender, race e education sono trattati come se fossero una categoria a parte, in age e income sono settati a 0)
train_df['age'] = train_df['age'].replace('unknown', '0')
val_df['age'] = val_df['age'].replace('unknown', '0')
dev_df['age'] = dev_df['age'].replace('unknown', '0')
train_df['income'] = train_df['income'].replace('unknown', '0')
val_df['income'] = val_df['income'].replace('unknown', '0')
dev_df['income'] = dev_df['income'].replace('unknown', '0')

train_df = train_df.astype({"gender": str, "education": str, "race": str, "age": int, "income": int})
val_df = val_df.astype({"gender": str, "education": str, "race": str, "age": int, "income": int})
dev_df = dev_df.astype({"gender": str, "education": str, "race": str, "age": int, "income": int})

emotions_encoder = EmotionsLabelEncoder()
emotions_encoder.fit(train_df.emotion)
y_train = emotions_encoder.encode(train_df.emotion)
y_val = emotions_encoder.encode(val_df.emotion)
y_dev = emotions_encoder.encode(dev_df.emotion)

#gender	education	race	age income
features_train =  np.array(train_df[['anger_count', 'disgust_count', 'fear_count', 'joy_count', 'sadness_count', 'surprise_count', 'hope_count']])
features_val =  np.array(val_df[['anger_count', 'disgust_count', 'fear_count', 'joy_count', 'sadness_count', 'surprise_count', 'hope_count']])
features_dev =  np.array(dev_df[['anger_count', 'disgust_count', 'fear_count', 'joy_count', 'sadness_count', 'surprise_count', 'hope_count']])

"""features_encoder = FeaturesEncoder()
features_encoder.fit(train_df[['gender', 'age']])
features_train = features_encoder.encode(train_df[['gender', 'age',]])
features_val = features_encoder.encode(val_df[['gender', 'age']])
features_dev = features_encoder.encode(dev_df[['gender', 'age']])"""

"features_encoder = FeaturesEncoder()\nfeatures_encoder.fit(train_df[['gender', 'age']])\nfeatures_train = features_encoder.encode(train_df[['gender', 'age',]])\nfeatures_val = features_encoder.encode(val_df[['gender', 'age']])\nfeatures_dev = features_encoder.encode(dev_df[['gender', 'age']])"

In [7]:
num_features=features_train.shape[1]

class BERTClassifier(BertPreTrainedModel):
    #torch.nn.Module):
    def __init__(self, config):#, num_features):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.num_features = num_features
        self.config = config

        self.bert = BertModel(config)
        #self.bert = BertModel.from_pretrained('bert-base-uncased')
        """classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)"""
        self.dropout = torch.nn.Dropout(0.3)
        #self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768+num_features, self.num_labels)
        self.post_init()
        """classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )"""
        """def __init__(self, num_labels, num_features):
        super().__init__(num_labels)
        self.num_labels = num_labels
        #super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768+num_features, self.num_labels)
        """
    def forward(self, input_ids, attention_mask=None,  
                token_type_ids=None, labels=None, 
                features=None, output_attentions=False):
        outputs = self.bert(
          input_ids,
          attention_mask=attention_mask,
          token_type_ids=token_type_ids,
          output_attentions=True
        ) # outputs type is BaseModelOutputWithPoolingAndCrossAttentions
        # dropout_output = self.dropout(outputs[0]) # outputs[0]=last hidden state, with shape (batch_size, sequence_length, hidden_size)
        # logits = self.linear(dropout_output[:,0,:].view(-1,768)) # from batch_size,1,768 to batch size, 768
        dropout_output = self.dropout(outputs.pooler_output) # above the clf token there is linear and tanh
        if features is not None:
          dropout_output = torch.cat((dropout_output, features), dim=1)
        logits = self.linear(dropout_output)
        loss = None
        if labels is not None:
            #loss = BCEWithLogitsLoss()(logits, labels)
            loss_fct = BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)
        return SequenceClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [8]:
############################ SUB-SAMPLE ############################
train_df = train_df
val_df = val_df
dev_df = dev_df
####################################################################

train_set = EMODataset(tokenizer=tokenizer, essay=train_df.essay, targets=y_train, features=features_train)
val_set = EMODataset(tokenizer=tokenizer, essay=val_df.essay, targets=y_val, features=features_val)
dev_set = EMODataset(tokenizer=tokenizer, essay=dev_df.essay, targets=y_dev, features=features_dev)

config = PretrainedConfig(pretrained_model_name_or_path=model_config['model_name'], 
                          num_labels=NUM_LABELS, 
                          ignore_mismatched_sizes=True, 
                          problem_type="multi_label_classification")
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model = BERTClassifier.from_pretrained('bert-base-uncased', num_labels=NUM_LABELS)#, num_features=features_train.shape[1])
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BERTClassifier: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BERTClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BERTClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BERTClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.bias', 'linear.weight']
You should prob

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [ ]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)))


for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
          batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(**batch)
          loss = outputs.loss
          loss.backward()

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar_train.update(1)

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar_eval.update(1)
    
  print(metric.compute())

      

In [ ]:
model.eval()

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=32, collate_fn=data_collator
)

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

In [9]:
train_arguments = TrainingArguments(
    output_dir="./",
    per_device_train_batch_size=model_config['train_batch_size'],
    per_device_eval_batch_size=model_config['val_batch_size'],
    num_train_epochs=model_config['epochs'],
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=model_config['learning_rate'],
    weight_decay=model_config['weight_decay'],
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    seed=model_config['seed'],
    #eval_accumulation_steps=10
) # TODO: custom other params

trainer = Trainer(
    model=model,
    args=train_arguments,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    compute_metrics=compute_EMO_metrics_trainer
)

class TrainerLoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path

    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero: # whether this process is the main one in a distributed setting
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")

trainer.add_callback(EarlyStoppingCallback(
    early_stopping_patience=model_config['patience'],
    early_stopping_threshold=model_config['early_stopping_threshold']))
trainer.add_callback(TrainerLoggingCallback(model_config['model_id']+"_log.json"))

In [10]:
"""import torch
torch.cuda.empty_cache()"""
#!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:256'

In [12]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_27/4032920361.py:1 in <module>                                                    │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_27/4032920361.py'                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2035 in _inner_training_loop     │
│                                                                                                  │
│   2032 │   │   │   │   self.control.should_training_stop = True                                  │
│   2033 │   │   │                                                                                 │
│   2034 │   │   │   self.control = self.callback_handler.on_epoch_end(args, self.state, self.con  │
│ ❱ 2035 │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_  │
│   2036 │   │   │                                                                                 │
│   2037 │   │   │   if DebugOption.TPU_METRICS_DEBUG in self.args.debug:                          │
│   2038 │   │   │   │   if is_torch_tpu_available():                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2321 in _maybe_log_save_evaluate │
│                                                                                                  │
│   2318 │   │   │   │   │   )                                                                     │
│   2319 │   │   │   │   │   metrics.update(dataset_metrics)                                       │
│   2320 │   │   │   else:                                                                         │
│ ❱ 2321 │   │   │   │   metrics = self.evaluate(ignore_keys=ignore_keys_for_eval)                 │
│   2322 │   │   │   self._report_to_hp_search(trial, self.state.global_step, metrics)             │
│   2323 │   │   │                                                                                 │
│   2324 │   │   │   # Run delayed LR scheduler now that metrics are populated                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3053 in evaluate                 │
│                                                                                                  │
│   3050 │   │   start_time = time.time()                                                          │
│   3051 │   │                                                                                     │
│   3052 │   │   eval_loop = self.prediction_loop if self.arg

In [ ]:
plot_loss_curve(train_loss_values, eval_loss_values, "loss", f"{model_config['model_name']}_loss.png")

In [ ]:
model = BERTClassifier(num_labels=NUM_LABELS, num_features=0)
input_str = "Hello world!"
plot_attentions(input_str, model, tokenizer, "./attention.pdf", "bert-base-cased")

In [ ]:
# Define your gold labels
gold_emotions = ['Hope/Sadness', 'Anger', 'Sadness', 'Neutral', 'Disgust/Sadness',
       'Anger/Disgust', 'Fear/Sadness', 'Joy', 'Hope', 'Joy/Neutral',
       'Disgust', 'Neutral/Sadness', 'Neutral/Surprise', 'Anger/Neutral',
       'Hope/Neutral', 'Surprise', 'Anger/Sadness', 'Fear', 'Anger/Joy',
       'Disgust/Fear', 'Fear/Neutral', 'Fear/Hope', 'Joy/Sadness',
       'Anger/Disgust/Sadness', 'Anger/Surprise', 'Disgust/Neutral',
       'Anger/Fear', 'Sadness/Surprise', 'Disgust/Surprise', 'Anger/Hope']

# Define the desired label names
label_names = ['Anger', 'Disgust', 'Fear', 'Hope', 'Joy', 'Neutral', 'Sadness', 'Surprise']

# Compute the confusion matrix
cm = confusion_matrix(gold_emotions, gold_emotions, labels=label_names)

# Plot the confusion matrix
fig, ax = plt.subplots()
im = ax.imshow(cm, cmap='Blues')

# Customize the plot
ax.set_xticks(np.arange(len(label_names)))
ax.set_yticks(np.arange(len(label_names)))
ax.set_xticklabels(label_names, rotation=45)
ax.set_yticklabels(label_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")

# Add colorbar
cbar = ax.figure.colorbar(im, ax=ax)

# Show the plot
plt.tight_layout()
plt.show()